## 币安做空超涨做多超跌策略的风险估计

先看原始报告：https://www.fmz.com/digest-topic/5294 ， 改进报告：https://www.fmz.com/digest-topic/5364

策略公开了4天，前期表现实在太好，收益很高，极少回撤，以至于很多人上了很高的杠杆，去搏每天10%的收益。但如同开始的报告所说，没有完美的策略，空超涨做多超跌是利用了山寨币同涨同跌的特性，如果有币种走出独特的趋势，会积累很多仓位，虽然改进版加上了移动平均来追踪从初始价格，但风险依然存在，本报告主要量化具体的风险，以及为什么参数推荐trade_value要占总资金的3%。

为了代码突出，把这部分提前了，大家先运行后面的代码（从导入库开始）。

为了模拟我们假设了20个币种，但只用添加BTC和ETH，用BTC代表价格不变的19个币，ETH表示出现独立行情的币种。由于只是模拟，这里就不用移动平均来追踪初始价格了，假设价格是很快的速度上涨来的。

首先模拟一下单个币价不断上涨的情况，stop_loss表示止损偏离，这里仅仅模拟，实际情况会有间歇性的回调，不会如此差。

假设这个币没有任何回调，但止损偏离为0.41时，此时ETH上涨了44%，最终亏损了7倍的交易额，即trade_value\*7。如果trade_value设置为总资金的3%，那么亏损=总资金\*0.03\*7。最大回撤约为0.03\*7=21%。

可以根据下面的结果估算自己的风险承受能力。

In [49]:
btc_price = [1]*500 #比特币价格，一直不变
eth_price = [i/100. for i in range(100,500)] #以太坊，一个周期上涨1%

for stop_loss in [i/1000. for i in range(10,1500,50)]:
    e = Exchange(['BTC','ETH'],initial_balance=10000,commission=0.0005,log=False)
    trade_value  = 300 # 交易额为300
    
    for i in range(200):

        index = (btc_price[i]*19+eth_price[i])/20. #指数

        e.Update(i,{'BTC':btc_price[i], 'ETH':eth_price[i]}) 

        diff_btc = btc_price[i] - index #偏差
        diff_eth = eth_price[i] - index

        btc_value = e.account['BTC']['value']*np.sign(e.account['BTC']['amount'])
        eth_value = e.account['ETH']['value']*np.sign(e.account['ETH']['amount'])

        aim_btc_value = -trade_value*round(diff_btc/0.01,1)*19 #这里BTC代替的是19个币种
        aim_eth_value = -trade_value*round(diff_eth/0.01,1)

        if aim_btc_value - btc_value > 20:
            e.Buy('BTC',btc_price[i],(aim_btc_value - btc_value)/btc_price[i])

        if aim_eth_value - eth_value < -20 and diff_eth < stop_loss:
            e.Sell('ETH',eth_price[i], (eth_value-aim_eth_value)/eth_price[i],diff_eth)

        if diff_eth > stop_loss and eth_value < 0: #止损
            stop_price = eth_price[i]
            e.Buy('ETH',eth_price[i], (-eth_value)/eth_price[i],diff_eth)

    print('币价：',stop_price,' 止损偏离：', stop_loss,'最终余额：',e.df['total'].iloc[-1], ' 亏损交易额倍数：',round((e.initial_balance-e.df['total'].iloc[-1])/300,1))

币价： 1.02  止损偏离： 0.01 最终余额： 9968.840396  亏损交易额倍数： 0.1
币价： 1.07  止损偏离： 0.06 最终余额： 9912.862738  亏损交易额倍数： 0.3
币价： 1.12  止损偏离： 0.11 最终余额： 9793.616067  亏损交易额倍数： 0.7
币价： 1.17  止损偏离： 0.16 最终余额： 9617.477263  亏损交易额倍数： 1.3
币价： 1.23  止损偏离： 0.21 最终余额： 9337.527299  亏损交易额倍数： 2.2
币价： 1.28  止损偏离： 0.26 最终余额： 9051.5166  亏损交易额倍数： 3.2
币价： 1.33  止损偏离： 0.31 最终余额： 8721.285267  亏损交易额倍数： 4.3
币价： 1.38  止损偏离： 0.36 最终余额： 8350.582251  亏损交易额倍数： 5.5
币价： 1.44  止损偏离： 0.41 最终余额： 7856.720861  亏损交易额倍数： 7.1
币价： 1.49  止损偏离： 0.46 最终余额： 7406.412066  亏损交易额倍数： 8.6
币价： 1.54  止损偏离： 0.51 最终余额： 6923.898356  亏损交易额倍数： 10.3
币价： 1.59  止损偏离： 0.56 最终余额： 6411.276143  亏损交易额倍数： 12.0
币价： 1.65  止损偏离： 0.61 最终余额： 5758.736222  亏损交易额倍数： 14.1
币价： 1.7  止损偏离： 0.66 最终余额： 5186.230956  亏损交易额倍数： 16.0
币价： 1.75  止损偏离： 0.71 最终余额： 4588.802975  亏损交易额倍数： 18.0
币价： 1.81  止损偏离： 0.76 最终余额： 3841.792751  亏损交易额倍数： 20.5
币价： 1.86  止损偏离： 0.81 最终余额： 3193.215479  亏损交易额倍数： 22.7
币价： 1.91  止损偏离： 0.86 最终余额： 2525.155765  亏损交易额倍数： 24.9
币价： 1.96  止损偏离： 0.91 最终余额： 1837.699982  亏

在模拟一下不断下跌的情况，下跌伴随这合约价值的减小，因此风险比上涨更高，随着价格下跌，亏损上升的速度加速。止损偏离值为-0.31时，此时币价下跌33%，亏损了6.5个交易额，如果交易额trade_value设置为总资金的3%，最大回撤约为0.03*6.5=19.5%。

In [56]:
btc_price = [1]*500 #比特币价格，一直不变
eth_price = [2-i/100. for i in range(100,200)] #以太坊

for stop_loss in [-i/1000. for i in range(10,1000,50)]:
    e = Exchange(['BTC','ETH'],initial_balance=10000,commission=0.0005,log=False)
    trade_value  = 300 # 交易额为300
    
    for i in range(100):

        index = (btc_price[i]*19+eth_price[i])/20. #指数

        e.Update(i,{'BTC':btc_price[i], 'ETH':eth_price[i]}) 

        diff_btc = btc_price[i] - index #偏差
        diff_eth = eth_price[i] - index

        btc_value = e.account['BTC']['value']*np.sign(e.account['BTC']['amount'])
        eth_value = e.account['ETH']['value']*np.sign(e.account['ETH']['amount'])

        aim_btc_value = -trade_value*round(diff_btc/0.01,1)*19 #这里BTC代替的是19个币种
        aim_eth_value = -trade_value*round(diff_eth/0.01,1)
        
        if aim_btc_value - btc_value < -20:
            e.Sell('BTC',btc_price[i],-(aim_btc_value - btc_value)/btc_price[i])
    
        if aim_eth_value - eth_value > 20 and diff_eth > stop_loss:
            e.Buy('ETH',eth_price[i], -(eth_value-aim_eth_value)/eth_price[i],diff_eth)

        if diff_eth < stop_loss and eth_value > 0:
            e.Sell('ETH',eth_price[i], (eth_value)/eth_price[i],diff_eth)
            stop_price = eth_price[i]
        
    print('币价：',round(stop_price,2),' 止损偏离：', stop_loss,'最终余额：',e.df['total'].iloc[-1], ' 亏损交易额倍数：',round((e.initial_balance-e.df['total'].iloc[-1])/300,1))

币价： 0.98  止损偏离： -0.01 最终余额： 9983.039091  亏损交易额倍数： 0.1
币价： 0.93  止损偏离： -0.06 最终余额： 9922.200148  亏损交易额倍数： 0.3
币价： 0.88  止损偏离： -0.11 最终余额： 9778.899361  亏损交易额倍数： 0.7
币价： 0.83  止损偏离： -0.16 最终余额： 9545.316075  亏损交易额倍数： 1.5
币价： 0.77  止损偏离： -0.21 最终余额： 9128.800213  亏损交易额倍数： 2.9
币价： 0.72  止损偏离： -0.26 最终余额： 8651.260863  亏损交易额倍数： 4.5
币价： 0.67  止损偏离： -0.31 最终余额： 8037.598952  亏损交易额倍数： 6.5
币价： 0.62  止损偏离： -0.36 最终余额： 7267.230651  亏损交易额倍数： 9.1
币价： 0.56  止损偏离： -0.41 最终余额： 6099.457595  亏损交易额倍数： 13.0
币价： 0.51  止损偏离： -0.46 最终余额： 4881.767442  亏损交易额倍数： 17.1
币价： 0.46  止损偏离： -0.51 最终余额： 3394.414792  亏损交易额倍数： 22.0
币价： 0.41  止损偏离： -0.56 最终余额： 1575.135344  亏损交易额倍数： 28.1
币价： 0.35  止损偏离： -0.61 最终余额： -1168.50508  亏损交易额倍数： 37.2
币价： 0.29  止损偏离： -0.66 最终余额： -4071.007983  亏损交易额倍数： 46.9
币价： 0.25  止损偏离： -0.71 最终余额： -7750.361195  亏损交易额倍数： 59.2
币价： 0.19  止损偏离： -0.76 最终余额： -13618.366286  亏损交易额倍数： 78.7
币价： 0.14  止损偏离： -0.81 最终余额： -20711.473968  亏损交易额倍数： 102.4
币价： 0.09  止损偏离： -0.86 最终余额： -31335.965608  亏损交易额倍数： 137.8
币价： 0.04

In [1]:
# 需要导入的库
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
price_usdt = pd.read_csv('https://www.fmz.com/upload/asset/20227de6c1d10cb9dd1.csv ', index_col = 0)
price_usdt.index = pd.to_datetime(price_usdt.index)
price_usdt_norm = price_usdt/price_usdt.fillna(method='bfill').iloc[0,]
price_usdt_btc = price_usdt.divide(price_usdt['BTC'],axis=0)
price_usdt_btc_norm = price_usdt_btc/price_usdt_btc.fillna(method='bfill').iloc[0,]

In [3]:
class Exchange:
    
    def __init__(self, trade_symbols, leverage=20, commission=0.00005,  initial_balance=10000, log=False):
        self.initial_balance = initial_balance #初始的资产
        self.commission = commission
        self.leverage = leverage
        self.trade_symbols = trade_symbols
        self.date = ''
        self.log = log
        self.df = pd.DataFrame(columns=['margin','total','leverage','realised_profit','unrealised_profit'])
        self.account = {'USDT':{'realised_profit':0, 'margin':0, 'unrealised_profit':0, 'total':initial_balance, 'leverage':0, 'fee':0}}
        for symbol in trade_symbols:
            self.account[symbol] = {'amount':0, 'hold_price':0, 'value':0, 'price':0, 'realised_profit':0, 'margin':0, 'unrealised_profit':0,'fee':0}
            
    def Trade(self, symbol, direction, price, amount, msg=''):
        if self.date and self.log:
            print('%-20s%-5s%-5s%-10.8s%-8.6s %s'%(str(self.date), symbol, 'buy' if direction == 1 else 'sell', price, amount, msg))
            
        cover_amount = 0 if direction*self.account[symbol]['amount'] >=0 else min(abs(self.account[symbol]['amount']), amount)
        open_amount = amount - cover_amount
        
        self.account['USDT']['realised_profit'] -= price*amount*self.commission #扣除手续费
        self.account['USDT']['fee'] += price*amount*self.commission
        self.account[symbol]['fee'] += price*amount*self.commission
        
        if cover_amount > 0: #先平仓
            self.account['USDT']['realised_profit'] += -direction*(price - self.account[symbol]['hold_price'])*cover_amount  #利润
            self.account['USDT']['margin'] -= cover_amount*self.account[symbol]['hold_price']/self.leverage #释放保证金
            
            self.account[symbol]['realised_profit'] += -direction*(price - self.account[symbol]['hold_price'])*cover_amount
            self.account[symbol]['amount'] -= -direction*cover_amount
            self.account[symbol]['margin'] -=  cover_amount*self.account[symbol]['hold_price']/self.leverage
            self.account[symbol]['hold_price'] = 0 if self.account[symbol]['amount'] == 0 else self.account[symbol]['hold_price']
            
        if open_amount > 0:
            total_cost = self.account[symbol]['hold_price']*direction*self.account[symbol]['amount'] + price*open_amount
            total_amount = direction*self.account[symbol]['amount']+open_amount
            
            self.account['USDT']['margin'] +=  open_amount*price/self.leverage            
            self.account[symbol]['hold_price'] = total_cost/total_amount
            self.account[symbol]['amount'] += direction*open_amount
            self.account[symbol]['margin'] +=  open_amount*price/self.leverage
            
        self.account[symbol]['unrealised_profit'] = (price - self.account[symbol]['hold_price'])*self.account[symbol]['amount']
        self.account[symbol]['price'] = price
        self.account[symbol]['value'] = abs(self.account[symbol]['amount'])*price
        
        return True
    
    def Buy(self, symbol, price, amount, msg=''):
        self.Trade(symbol, 1, price, amount, msg)
        
    def Sell(self, symbol, price, amount, msg=''):
        self.Trade(symbol, -1, price, amount, msg)
        
    def Update(self, date, close_price): #对资产进行更新
        self.date = date
        self.close = close_price
        self.account['USDT']['unrealised_profit'] = 0
        for symbol in self.trade_symbols:
            if np.isnan(close_price[symbol]):
                continue
            self.account[symbol]['unrealised_profit'] = (close_price[symbol] - self.account[symbol]['hold_price'])*self.account[symbol]['amount']
            self.account[symbol]['price'] = close_price[symbol]
            self.account[symbol]['value'] = abs(self.account[symbol]['amount'])*close_price[symbol]
            self.account['USDT']['unrealised_profit'] += self.account[symbol]['unrealised_profit']
        
        self.account['USDT']['total'] = round(self.account['USDT']['realised_profit'] + self.initial_balance + self.account['USDT']['unrealised_profit'],6)
        self.account['USDT']['leverage'] = round(self.account['USDT']['margin']/self.account['USDT']['total'],4)*self.leverage
        self.df.loc[self.date] = [self.account['USDT']['margin'],self.account['USDT']['total'],self.account['USDT']['leverage'],self.account['USDT']['realised_profit'],self.account['USDT']['unrealised_profit']]